# Accesing Watsonx.ai via REST API

In this lab, we will take a comprehensive look at making HTTP requests to access [Watsonx.ai's REST API](https://workbench.res.ibm.com/docs/api-reference) and learn how to use the functionality.  This lab explore only a few of the many REST endpoints available so explore the REST API documentation to view the full list of capabilities.

In [55]:
from dotenv import load_dotenv
from ibm_cloud_sdk_core import IAMTokenManager
import json
import os
import requests

## HTTP request headers
Headers contain parameter values that represent the metadata associated with an API requests and response. In the following example, the Authorization header provides the server with credentials to validate your access.  Watsonx.ai uses a "Bearer" access token wich is used to pass our Watsonx.ai authentication key.  The 'Content-type' header in the request is added to tell the server or the browser which is serving the resource to the end user about the media type of the request. In this case, type of expected data as 'application/json'.

In [56]:
load_dotenv()
api_key = os.getenv("API_KEY", None)
ibm_cloud_url= os.getenv("IBM_CLOUD_URL", None)
project_id= os.getenv("PROJECT_ID", None)
if api_key is None or ibm_cloud_url is None or project_id is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    access_token = IAMTokenManager(
        apikey = api_key,
        url = "https://iam.cloud.ibm.com/identity/token").get_token()       
    headers = {
            "Authorization": "Bearer " + access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

## POST vs GET
HTTP requests come in two flavors: GET and POST.  When using GET, data parameters are included in the URL and visible to everyone. However, when using POST, data is not displayed in the URL but is instead passed in the HTTP message body. 

GET requests are intended to retrieve data from a server and do not modify the server’s state. On the other hand, POST requests are used to send data to the server for processing and may modify the server’s state.  

## POST requests with 'Generate' endpoint

The generate endpoint "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text" provides an interface for sending prompts to any model supported by Watsonx.ai. Given a text prompt as inputs, and required parameters, the selected model will attempt to complete the provide input and return "generated_text".

Request body needs to include:
- Model id (string): the id of the model
- Input (string): prompt to generate completion
- Parameters for the model (key-value pairs)
- your watsonx project ID


In [57]:
body={
  "model_id": "google/flan-ul2",
  "input": "Write a short blog post for an advanced cloud service for large language models: This service is",
  "parameters": {
    "temperature": 0,
    "max_new_tokens": 50,
    "min_new_tokens": 25
  },
  "project_id":project_id
}

In [58]:
generation_endpoint = ibm_cloud_url + "/ml/v1-beta/generation/text?version=2023-07-07"
response=requests.post(url=generation_endpoint, headers=headers, json=body )
print("JSON Response: ",response.json())


JSON Response:  {'errors': [{'code': 'token_quota_reached', 'message': 'Request of 1 token(s) from quota was rejected'}], 'trace': '9c52c1860914888ad12479a674468237', 'status_code': 403}


## Using GET requests to retrieve data
The GET method is used to retrieve information from Watsonx.ai using a given URL.

### GET /models
Get the list of all models supported by Watsonx.ai

In [52]:
model_endpoint = ibm_cloud_url + "/ml/v1-beta/foundation_model_specs?version=2023-07-07"
response = requests.get(url = model_endpoint, headers=headers)
models = response.json()['resources']

print(f"{len(models)} models supported the Watsonx.ai")

# Print the first two models
models[0:2]

5 models supported the Watsonx.ai


[{'model_id': 'bigscience/mt0-xxl',
  'label': 'mt0-xxl-13b',
  'provider': 'BigScience',
  'source': 'Hugging Face',
  'short_description': 'An instruction-tuned iteration on mT5.',
  'long_description': 'mt0-xxl (13B) is an instruction-tuned iteration on mT5. Like BLOOMZ, it was fine-tuned on a cross-lingual task mixture dataset (xP3) using multitask prompted finetuning (MTF).',
  'task_ids': ['question_answering',
   'summarization',
   'classification',
   'generation'],
  'tasks': [{'id': 'question_answering', 'ratings': {'quality': 3}},
   {'id': 'summarization', 'ratings': {'quality': 3}},
   {'id': 'classification', 'ratings': {'quality': 3}},
   {'id': 'extraction', 'ratings': {'quality': 2}}],
  'limits': {'lite': {'call_time': '5m0s',
    'max_output_tokens': 700,
    'max_input_tokens': 0},
   'v2-professional': {'call_time': '5m0s',
    'max_output_tokens': 700,
    'max_input_tokens': 0},
   'v2-standard': {'call_time': '5m0s',
    'max_output_tokens': 700,
    'max_input

### Lab Complete